In [1]:
from google.colab import drive

drive.mount("/content/gdrive")

Mounted at /content/gdrive


In [2]:
%cd gdrive/My Drive

/content/gdrive/My Drive


In [3]:
import os 

if not os.path.isdir('neuro-sat'):
  !git clone --recurse-submodules https://github.com/dmeoli/neuro-sat
else:
  %cd neuro-sat
  !git pull

/content/gdrive/My Drive/neuro-sat
Already up to date.


In [4]:
!pip install -r requirements.txt

Looking in links: https://pytorch-geometric.com/whl/torch-1.9.0+cu102.html
     |████████████████████████████████| 124 kB 7.3 MB/s 
     |████████████████████████████████| 222 kB 57.2 MB/s 
     |████████████████████████████████| 1.4 MB 9.1 MB/s 
     |████████████████████████████████| 2.6 MB 54.4 MB/s 
     |████████████████████████████████| 376 kB 60.0 MB/s 
     |████████████████████████████████| 45 kB 4.1 MB/s 
  Created wheel for torch-geometric: filename=torch_geometric-1.7.2-py3-none-any.whl size=388142 sha256=9773d31fae67034c964157925665ff547c009e463b930ffce4d105f736963fc1
  Stored in directory: /root/.cache/pip/wheels/55/93/b6/2eeb0465afe89aee74d7a07a606e9770466d7565abd45a99d5
Successfully built torch-geometric


### Split dataset

In [ ]:
PROBLEM_TYPE="uniform-random-3-sat"
PROBLEM_NAME="uf50-218"

In [ ]:
!rm -rf ./data/"$PROBLEM_TYPE"/train
!rm -rf ./data/"$PROBLEM_TYPE"/val
!rm -rf ./data/"$PROBLEM_TYPE"/test

!splitfolders --output ./data/"$PROBLEM_TYPE" \
  --ratio 0.8 0.1 0.1 \
  --seed 123456 \
  -- ./data/"$PROBLEM_TYPE"

Copying files: 2101 files [13:58,  2.50 files/s]


# NeuroSAT

In [ ]:
%cd NeuroSAT

/content/gdrive/My Drive/neuroSAT/NeuroSAT-tf1


In [ ]:
!bash setup.sh

Reading package lists... Done
Building dependency tree       
Reading state information... Done
zlib1g-dev is already the newest version (1:1.2.11.dfsg-0ubuntu2).
zlib1g-dev set to manually installed.
The following additional packages will be installed:
  python-pip-whl python3-asn1crypto python3-cffi-backend python3-crypto
  python3-cryptography python3-idna python3-keyring python3-keyrings.alt
  python3-pkg-resources python3-secretstorage python3-setuptools python3-six
  python3-wheel python3-xdg
Suggested packages:
  python-crypto-doc python-cryptography-doc python3-cryptography-vectors
  gnome-keyring libkf5wallet-bin gir1.2-gnomekeyring-1.0
  python-secretstorage-doc python-setuptools-doc
The following NEW packages will be installed:
  python-pip-whl python3-asn1crypto python3-cffi-backend python3-crypto
  python3-cryptography python3-idna python3-keyring python3-keyrings.alt
  python3-pip python3-pkg-resources python3-secretstorage python3-setuptools
  python3-six python3-wheel p

In [ ]:
%cd ..

/content/gdrive/My Drive/neuroSAT


# AlphaZeroSAT

In [5]:
%cd AlphaZeroSAT

/content/gdrive/My Drive/neuro-sat/AlphaZeroSAT


### Build C++

In [6]:
%cd MCTSminisat
!apt install libgsl-dev
!sudo ln -s --force /usr/local/lib/python3.7/dist-packages/numpy/core/include/numpy /usr/include/numpy  # https://stackoverflow.com/a/44935933/5555994
!make distclean && CXXFLAGS=-w make && make python-wrap PYTHON=python3.7
!apt install swig
!swig -fastdispatch -c++ -python minisat/gym/GymSolver.i
%cd ..

/content/gdrive/My Drive/neuro-sat/AlphaZeroSAT/MCTSminisat
Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following additional packages will be installed:
  libgsl23 libgslcblas0
Suggested packages:
  gsl-ref-psdoc | gsl-doc-pdf | gsl-doc-info | gsl-ref-html
The following NEW packages will be installed:
  libgsl-dev libgsl23 libgslcblas0
0 upgraded, 3 newly installed, 0 to remove and 40 not upgraded.
Need to get 1,926 kB of archives.
After this operation, 9,474 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu bionic/universe amd64 libgslcblas0 amd64 2.4+dfsg-6 [79.7 kB]
Get:2 http://archive.ubuntu.com/ubuntu bionic/universe amd64 libgsl23 amd64 2.4+dfsg-6 [823 kB]
Get:3 http://archive.ubuntu.com/ubuntu bionic/universe amd64 libgsl-dev amd64 2.4+dfsg-6 [1,023 kB]
Fetched 1,926 kB in 1s (1,370 kB/s)
Selecting previously unselected package libgslcblas0:amd64.
(Reading database ... 160837 files and direct

In [7]:
!python train.py --help

train.py:86: SyntaxWarning: assertion is always true, perhaps remove parentheses?
  assert (len(self.ev_hist) <= self.length_hist,
train.py:322: SyntaxWarning: assertion is always true, perhaps remove parentheses?
  assert (args.save_dir is not None and model_dir is not None,
2021-07-21 15:49:36.978335: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcudart.so.11.0
usage: train.py [-h] [--save_dir SAVE_DIR] [--best_model BEST_MODEL]
                [--status_file STATUS_FILE] [--result_file RESULT_FILE]
                [--dump_dir DUMP_DIR] [--dump_file DUMP_FILE]
                [--train_path TRAIN_PATH] [--test_path TEST_PATH]
                [--max_clause MAX_CLAUSE] [--max_var MAX_VAR]
                [--sl_buffer_size SL_BUFFER_SIZE] [--n_batch N_BATCH]
                [--n_stack N_STACK] [--n_repeat N_REPEAT] [--n_start N_START]
                [--n_train_files N_TRAIN_FILES] [--n_test_files N_TEST_FILES]
                [--l

In [17]:
!python train.py \
  --save_dir='runs/uf50-218' \
  --best_model=-1 \
  --status_file='status.pkl' \
  --result_file='result.pkl' \
  --dump_dir='parameters' \
  --dump_file='sl.pkl' \
  --train_path='../data/uniform-random-3-sat/train/uf50-218' \
  --test_path='../data/uniform-random-3-sat/test/uf50-218' \
  --max_clause=120 \
  --max_var=20 \
  --sl_buffer_size=1000000 \
  --n_batch=32 \
  --n_stack=1 \
  --n_repeat=100 \
  --n_start=0 \
  --n_train_files=0 \
  --n_test_files=0 \
  --l2_coeff=0.0001 \
  --sl_num_steps=64000 \
  --sl_n_batch=32 \
  --sl_n_checkpoint=32000 \
  --n_cycles=2 \
  --show_only='No' \
  --which_cycle=0

2021-07-21 16:19:06.669525: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcudart.so.11.0
best_model is 0
n_start is 32
ev_hist has length 0
length_hist is 1
status_file is runs/uf50-218/status.pkl
args is runs/uf50-218__../data/uniform-random-3-sat/train/uf50-218__../data/uniform-random-3-sat/test/uf50-218
2021-07-21 16:19:08.426045: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcuda.so.1
2021-07-21 16:19:08.440754: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2021-07-21 16:19:08.441353: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1733] Found device 0 with properties: 
pciBusID: 0000:00:04.0 name: Tesla P100-PCIE-16GB computeCapability: 6.0
coreClock: 1.3285GHz coreCount: 56 deviceMemorySize: 15.90GiB deviceMemoryBandwidth

In [ ]:
%cd ..

/content/gdrive/My Drive/neuro-sat


# GraphQSAT

In [ ]:
%cd GraphQSAT

/content/gdrive/My Drive/neuro-sat/GraphQSAT


### Build C++

In [ ]:
%cd minisat
!make distclean && CXXFLAGS=-w make && make python-wrap PYTHON=python3.7
!apt install swig
!swig -fastdispatch -c++ -python minisat/gym/GymSolver.i
%cd ..

/content/gdrive/My Drive/neuro-sat/GraphQSAT/minisat
rm -f   build/release/minisat/core/Solver.o  build/release/minisat/core/Main.o  build/release/minisat/simp/SimpSolver.o  build/release/minisat/simp/Main.o  build/release/minisat/utils/System.o  build/release/minisat/utils/Options.o  build/release/minisat/gym/GymSolver.o   build/debug/minisat/core/Solver.o  build/debug/minisat/core/Main.o  build/debug/minisat/simp/SimpSolver.o  build/debug/minisat/simp/Main.o  build/debug/minisat/utils/System.o  build/debug/minisat/utils/Options.o  build/debug/minisat/gym/GymSolver.o   build/profile/minisat/core/Solver.o  build/profile/minisat/core/Main.o  build/profile/minisat/simp/SimpSolver.o  build/profile/minisat/simp/Main.o  build/profile/minisat/utils/System.o  build/profile/minisat/utils/Options.o  build/profile/minisat/gym/GymSolver.o   build/dynamic/minisat/core/Solver.o  build/dynamic/minisat/core/Main.o  build/dynamic/minisat/simp/SimpSolver.o  build/dynamic/minisat/simp/Main.o  build/dyna

### Add metadata for evaluation (train and validation set)

In [ ]:
!python add_metadata.py --eval-problems-paths ../data/$PROBLEM_TYPE/train/$PROBLEM_NAME
!python add_metadata.py --eval-problems-paths ../data/$PROBLEM_TYPE/val/$PROBLEM_NAME

[Errno 2] No such file or directory: '/content/gdrive/MyDrive/neuroSAT/data/uniform-random-3-sat/train/uf50-218/METADATA'
No metadata available, that is fine for metadata generator.
Testing agent <MiniSAT Agent>... with_restarts is set to False
Rollout 1, steps 44, num_restarts 0.
Rollout 2, steps 24, num_restarts 0.
Rollout 3, steps 13, num_restarts 0.
Rollout 4, steps 24, num_restarts 0.
Rollout 5, steps 32, num_restarts 0.
Rollout 6, steps 13, num_restarts 0.
Rollout 7, steps 35, num_restarts 0.
Rollout 8, steps 58, num_restarts 0.
Rollout 9, steps 58, num_restarts 0.
Rollout 10, steps 15, num_restarts 0.
Rollout 11, steps 53, num_restarts 0.
Rollout 12, steps 23, num_restarts 0.
Rollout 13, steps 11, num_restarts 0.
Rollout 14, steps 29, num_restarts 0.
Rollout 15, steps 12, num_restarts 0.
Rollout 16, steps 26, num_restarts 0.
Rollout 17, steps 32, num_restarts 0.
Rollout 18, steps 47, num_restarts 0.
Rollout 19, steps 21, num_restarts 0.
Rollout 20, steps 84, num_restarts 0.
Roll

### Train

In [ ]:
!python dqn.py --help

usage: dqn.py [-h] [--lr LR] [--bsize BSIZE] [--eps-init EPS_INIT]
              [--expert-exploration-prob EXPERT_EXPLORATION_PROB]
              [--gamma GAMMA] [--eps-decay-steps EPS_DECAY_STEPS]
              [--target-update-freq TARGET_UPDATE_FREQ]
              [--batch-updates BATCH_UPDATES] [--buffer-size BUFFER_SIZE]
              [--history-len HISTORY_LEN]
              [--init-exploration-steps INIT_EXPLORATION_STEPS]
              [--step-freq STEP_FREQ] [--loss LOSS] [--save-freq SAVE_FREQ]
              [--train-problems-paths TRAIN_PROBLEMS_PATHS]
              [--eval-freq EVAL_FREQ] [--eval-time-limit EVAL_TIME_LIMIT]
              [--status-dict-path STATUS_DICT_PATH] [--core-steps CORE_STEPS]
              [--priority_alpha PRIORITY_ALPHA]
              [--priority_beta PRIORITY_BETA] [--opt OPT]
              [--e2v-aggregator E2V_AGGREGATOR] [--n_hidden N_HIDDEN]
              [--hidden_size HIDDEN_SIZE]
              [--decoder_v_out_size DECODER_V_OUT_SIZE]
   

In [ ]:
!python dqn.py \
  --logdir ./log \
  --env-name sat-v0 \
  --train-problems-paths ../data/$PROBLEM_TYPE/train/$PROBLEM_NAME \
  --eval-problems-paths ../data/$PROBLEM_TYPE/val/$PROBLEM_NAME \
  --lr 0.00002 \
  --bsize 64 \
  --buffer-size 20000 \
  --eps-init 1.0 \
  --eps-final 0.01 \
  --gamma 0.99 \
  --eps-decay-steps 30000 \
  --batch-updates 50000 \
  --history-len 1 \
  --init-exploration-steps 5000 \
  --step-freq 4 \
  --target-update-freq 10 \
  --loss mse \
  --opt adam \
  --save-freq 500 \
  --grad_clip 1 \
  --grad_clip_norm_type 2 \
  --eval-freq 1000 \
  --eval-time-limit 3600 \
  --core-steps 4 \
  --expert-exploration-prob 0.0 \
  --priority_alpha 0.5 \
  --priority_beta 0.5 \
  --e2v-aggregator sum \
  --n_hidden 1 \
  --hidden_size 64 \
  --decoder_v_out_size 32 \
  --decoder_e_out_size 1 \
  --decoder_g_out_size 1 \
  --encoder_v_out_size 32 \
  --encoder_e_out_size 32 \
  --encoder_g_out_size 32 \
  --core_v_out_size 64 \
  --core_e_out_size 64 \
  --core_g_out_size 32 \
  --activation relu \
  --penalty_size 0.1 \
  --train_time_max_decisions_allowed 500 \
  --test_time_max_decisions_allowed 500 \
  --no_max_cap_fill_buffer \
  --lr_scheduler_gamma 1 \
  --lr_scheduler_frequency 3000 \
  --independent_block_layers 0

Streaming output truncated to the last 5000 lines.
Evaluation episode 84 is over. Your score is 1.9166666666666667.
It took 0.08026814460754395 seconds to solve a problem.
Evaluation episode 85 is over. Your score is 2.0.
It took 0.1082448959350586 seconds to solve a problem.
Evaluation episode 86 is over. Your score is 1.4736842105263157.
It took 0.08483314514160156 seconds to solve a problem.
Evaluation episode 87 is over. Your score is 2.5.
It took 0.04548811912536621 seconds to solve a problem.
Evaluation episode 88 is over. Your score is 8.0.
It took 0.08012032508850098 seconds to solve a problem.
Evaluation episode 89 is over. Your score is 3.5714285714285716.
It took 0.054953813552856445 seconds to solve a problem.
Evaluation episode 90 is over. Your score is 3.3.
It took 0.07018232345581055 seconds to solve a problem.
Evaluation episode 91 is over. Your score is 5.083333333333333.
It took 0.06575608253479004 seconds to solve a problem.
Evaluation episode 92 is over. Your score 

### Add metadata for evaluation (test set)

In [ ]:
!python add_metadata.py --eval-problems-paths ../data/$PROBLEM_TYPE/test/$PROBLEM_NAME

[Errno 2] No such file or directory: '/content/gdrive/MyDrive/neuroSAT/data/uniform-random-3-sat/test/uf50-218/METADATA'
No metadata available, that is fine for metadata generator.
Testing agent <MiniSAT Agent>... with_restarts is set to False
Rollout 1, steps 17, num_restarts 0.
Rollout 2, steps 56, num_restarts 0.
Rollout 3, steps 17, num_restarts 0.
Rollout 4, steps 11, num_restarts 0.
Rollout 5, steps 55, num_restarts 0.
Rollout 6, steps 47, num_restarts 0.
Rollout 7, steps 49, num_restarts 0.
Rollout 8, steps 29, num_restarts 0.
Rollout 9, steps 71, num_restarts 0.
Rollout 10, steps 28, num_restarts 0.
Rollout 11, steps 79, num_restarts 0.
Rollout 12, steps 16, num_restarts 0.
Rollout 13, steps 113, num_restarts 0.
Rollout 14, steps 23, num_restarts 0.
Rollout 15, steps 38, num_restarts 0.
Rollout 16, steps 48, num_restarts 0.
Rollout 17, steps 94, num_restarts 0.
Rollout 18, steps 17, num_restarts 0.
Rollout 19, steps 17, num_restarts 0.
Rollout 20, steps 32, num_restarts 0.
Roll

### Evaluate

In [ ]:
!python evaluate.py --help

usage: evaluate.py [-h] [--core-steps CORE_STEPS]
                   [--eval-time-limit EVAL_TIME_LIMIT] [--with_restarts]
                   [--no_restarts] [--compare_with_restarts]
                   [--compare_no_restarts]
                   [--test_max_data_limit_per_set TEST_MAX_DATA_LIMIT_PER_SET]
                   [--test_time_max_decisions_allowed TEST_TIME_MAX_DECISIONS_ALLOWED]
                   [--env-name ENV_NAME] [--debug] [--model-dir MODEL_DIR]
                   [--model-checkpoint MODEL_CHECKPOINT] [--logdir LOGDIR]
                   [--eps-final EPS_FINAL]
                   [--eval-problems-paths EVAL_PROBLEMS_PATHS]
                   [--train_max_data_limit_per_set TRAIN_MAX_DATA_LIMIT_PER_SET]
                   [--no-cuda] [--dump_timings_path DUMP_TIMINGS_PATH]

optional arguments:
  -h, --help            show this help message and exit
  --core-steps CORE_STEPS
                        Number of message passing iterations. \-1 for the same
                 

In [ ]:
!python evaluate.py \
  --logdir ./log \
  --env-name sat-v0 \
  --core-steps -1 \
  --eps-final 0.0 \
  --eval-time-limit 100000000000000 \
  --no_restarts \
  --test_time_max_decisions_allowed 500 \
  --eval-problems-paths ../data/$PROBLEM_TYPE/test/$PROBLEM_NAME \
  --model-dir ./runs/Jul13_15-58-05_7788e09a84e6/ \
  --model-checkpoint model_10000.chkp \
  >> ../data/$PROBLEM_TYPE/$PROBLEM_NAME/"$PROBLEM_NAME-gqsat.tsv"

In [ ]:
%cd ..